## Data enhancement

In [19]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### Pre-processing images in data
** note that only training data will undergo enhancement

#### Image enhancement
The process of improving the appearance of images to highlight specific features, reduce noise or improve the quality of the image, this helps it to be more suitable for analysis.

Alpha focuses on the contrast of image
- alpha greater than 1: images brighter, enhanced contrast
- alpha less than 1: images darker, reduced contrast

Beta focuses on brightness of image
- beta positive: makes images brighter
- beta negative: makes images darker

In [2]:
train_data = pd.read_csv("datasets/csv/train_data.csv")
test_data = pd.read_csv("datasets/csv/test_data.csv")

## PREPROCESSING IMAGES WITH ENHANCEMENT
def preprocess_image(img_path, enhance):
    # reading images
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if enhance == True:
        # enhancing image
        img = cv2.convertScaleAbs(img, alpha = 1.5, beta = -20)
    # target_size of 224, 224 commonly used for image classification
    img = cv2.resize(img, (224, 224))
    # normalising pixel values
    img_array = img.astype(np.float32) / 255
    return img_array

In [3]:
# using preprocessed images as train data
train_images = np.array([preprocess_image(image_path, True) for image_path in train_data["image_path"]])
# using "pathology" column as train labels
train_labels = np.array(train_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
train_labels[train_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

In [4]:
# using preprocessed images as test data
test_images = np.array([preprocess_image(image_path, False) for image_path in test_data["image_path"]])
# using "pathology" column as test labels
test_labels = np.array(test_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
test_labels[test_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

#### Image augmentation
The augmented image stores the following:
- original image without enhancement
- enhanced image with enhancement
- all combinations of augmented flips (with enhancement)
- all combinations of augmented flips (without enhancement)

In [5]:
## IMAGE AUGMENTATION - using numpy
augmented_images = []
augmented_labels = []

# augmenting images and storing in lists
for i, img_path in enumerate(train_data["image_path"]):
    original_image = preprocess_image(img_path, False)
    enhanced_image = preprocess_image(img_path, True)
    
    # making all combinations of flips
    for horizontal_flip in [True, False]:
        for vertical_flip in [True, False]:
            # applying flips on original image
            augmented_image = original_image
            if horizontal_flip:
                augmented_image = np.fliplr(augmented_image)
            if vertical_flip:
                augmented_image = np.flipud(augmented_image)
                
            # adding augmented image and label
            augmented_images.append(augmented_image)
            augmented_labels.append(train_labels[i])
            
            # applying flips on enhanced image
            augmented_image = enhanced_image
            if horizontal_flip:
                augmented_image = np.fliplr(augmented_image)
            if vertical_flip:
                augmented_image = np.flipud(augmented_image)
                
            # adding augmented image and label
            augmented_images.append(augmented_image)
            augmented_labels.append(train_labels[i])
        
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

print(len(train_labels) * 8)
print(len(augmented_labels))

18816
18816


In [6]:
# encoding labels
label_encoder = LabelEncoder()
encoded_train_labels = label_encoder.fit_transform(train_labels)
encoded_aug_train_labels = label_encoder.fit_transform(augmented_labels)
encoded_test_labels = label_encoder.fit_transform(test_labels)
# one-hot encode labels
one_hot_train_labels = tf.keras.utils.to_categorical(encoded_train_labels)
one_hot_aug_train_labels = tf.keras.utils.to_categorical(encoded_aug_train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(encoded_test_labels)

In [7]:
## ## VERSION 2 OF IMAGE AUGMENTATION - using TensorFlow
## datagen = ImageDataGenerator(
##     horizontal_flip = True,
##     vertical_flip = True,
##     fill_mode = "nearest"
## )
## 
## augmented_generator = datagen.flow(train_images[:, :, :, np.newaxis], one_hot_train_labels, batch_size = 32)

## Creating of models
- **Conv2D** performs convolutional operations on the input image data. It applies a set of filters to the input images to extract features.
- **MaxPooling2D** is a down-sampling operation that reduces the spatial dimensions, used after Conv2D layers to retain the most important information.
- **Flatten** is used to convert the multi-dimensional output of the previous laters into 1D.
- **Dense** represents a fully connected layer, where each neuron or node is connected to every neuron in the previous layer.

### Base model
Ensures that the data can be trained on.

In [8]:
base_model = Sequential()
# creating stack of Conv2D and MaxPooling2D
base_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
base_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
base_model.add(Flatten())
base_model.add(Dense(128, activation = "relu"))
# output layer with softmax
base_model.add(Dense(2, activation = "softmax"))

It was noted that whilst the adam optimizer had the highest accuracy, the nadam optimizer has a higher validation accuracy, which would suggest a change in choice to the preliminary report.

In [9]:
# compile model, improving accuracy
base_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = base_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
588/588 [==============================] - 445s 752ms/step - loss: 0.7211 - accuracy: 0.6105 - val_loss: 0.6284 - val_accuracy: 0.6480
Epoch 2/10
588/588 [==============================] - 440s 748ms/step - loss: 0.5543 - accuracy: 0.7021 - val_loss: 0.6315 - val_accuracy: 0.6480
Epoch 3/10
588/588 [==============================] - 442s 751ms/step - loss: 0.3836 - accuracy: 0.8254 - val_loss: 0.7155 - val_accuracy: 0.6514
Epoch 4/10
588/588 [==============================] - 444s 755ms/step - loss: 0.2150 - accuracy: 0.9210 - val_loss: 0.9113 - val_accuracy: 0.6395
Epoch 5/10
588/588 [==============================] - 444s 755ms/step - loss: 0.1098 - accuracy: 0.9709 - val_loss: 1.0695 - val_accuracy: 0.6310
Epoch 6/10
588/588 [==============================] - 443s 754ms/step - loss: 0.0726 - accuracy: 0.9862 - val_loss: 1.2130 - val_accuracy: 0.6395
Epoch 7/10
588/588 [==============================] - 442s 752ms/step - loss: 0.0620 - accuracy: 0.9891 - val_loss: 1.2193 -

### Improving with additional layers

In [12]:
twoLayer_model = Sequential()

# first convolutional layer
twoLayer_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
twoLayer_model.add(MaxPooling2D((2, 2)))

# second convolutional layer
twoLayer_model.add(Conv2D(64, (3, 3), activation = "relu"))
twoLayer_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
twoLayer_model.add(Flatten())
twoLayer_model.add(Dense(128, activation = "relu"))
# using dropout for regularisation (reduces overfitting)
twoLayer_model.add(Dropout(0.5))
# output layer with softmax
twoLayer_model.add(Dense(2, activation = "softmax"))

In [40]:
# compile model, improving accuracy
twoLayer_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = twoLayer_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

In [ ]:
threeLayer_model = Sequential()

# first convolutional layer
threeLayer_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
threeLayer_model.add(MaxPooling2D((2, 2)))

# second convolutional layer
threeLayer_model.add(Conv2D(64, (3, 3), activation = "relu"))
threeLayer_model.add(MaxPooling2D((2, 2)))

# third convolutional layer
threeLayer_model.add(Conv2D(128, (3, 3), activation = "relu"))
threeLayer_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
threeLayer_model.add(Flatten())
threeLayer_model.add(Dense(128, activation = "relu"))
# using dropout for regularisation (reduces overfitting)
threeLayer_model.add(Dropout(0.5))
# output layer with softmax
threeLayer_model.add(Dense(2, activation = "softmax"))

In [ ]:
# compile model, improving accuracy
threeLayer_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = threeLayer_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

### Hyperparameter tuning
To conduct hyperparameter tuning, it is beneficial to wrap the model in a function. This makes it scikit-learn compatable as there will be methods like Grid Search and Randomised Search to help optimise the performance of the neural network.

As more runs will be done, adding verbose = 2 makes the information printed a single line.

In [38]:
# defining model as a function
def create_3Layer_model(dropout_rate = 0.5, **kwargs):
    model = Sequential()

    # first convolutional layer
    model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
    model.add(MaxPooling2D((2, 2)))

    # second convolutional layer
    model.add(Conv2D(64, (3, 3), activation = "relu"))
    model.add(MaxPooling2D((2, 2)))

    # third convolutional layer
    model.add(Conv2D(128, (3, 3), activation = "relu"))
    model.add(MaxPooling2D((2, 2)))

    # unrolling output to 1D
    model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    # using dropout for regularisation (reduces overfitting)
    model.add(Dropout(dropout_rate))
    # output layer with softmax
    model.add(Dense(2, activation = "softmax"))
    
    # compile model, improving accuracy
    model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    # train model, validating on test set
    history = model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))
    
    return model, history

#### Conducting Grid Search
[modify]
Grid search involves defining a grid of hyperparameter values and training the model for each combination. It's a brute-force approach that explores a predefined set of hyperparameter values.

```
# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameters to search
param_grid = {
    'optimizer': ['adam', 'nadam', 'rmsprop'],
    'dropout_rate': [0.2, 0.4, 0.6],
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(augmented_images, one_hot_aug_train_labels)
```

# help

In [39]:
# creating the Keras classifier
model_for_grid = KerasClassifier(build_fn = create_3Layer_model, epochs = 10, batch_size = 32, verbose = 2)
# defining hyperparameters
parameter = {
    'dropout_rate': [0.2, 0.3, 0.4, 0.5]
}

# performing grid search - 5 fold grid search
grid = GridSearchCV(estimator = model_for_grid, param_grid = parameter)
grid_results = grid.fit(augmented_images, one_hot_aug_train_labels)

ValueError: Invalid parameter dropout_rate for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(dropout_rate=0.2)`
Check the list of available parameters with `estimator.get_params().keys()`

#### Conducting Random Search
[modify] Random search randomly samples hyperparameter values from predefined ranges. It is more efficient than grid search and can be effective in high-dimensional spaces.

```
from scipy.stats import uniform

# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameters and their distributions to sample from
param_dist = {
    'optimizer': ['adam', 'nadam', 'rmsprop'],
    'dropout_rate': uniform(0.2, 0.6),
}

# Perform random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3)
random_result = random_search.fit(augmented_images, one_hot_aug_train_labels)
```